问题1——手工实现PCA，特征向量降为16D

In [31]:
import numpy as np
import os
import cv2
import shutil
import math
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

#手工实现PCA
# X代表训练集矩阵
# k代表需要保留的特征数量（即降维后的特征向量数量）
def pca(X,k):
    temp = k
    #矩阵行数代表样本数量n_samples，矩阵列数代表每个样本的特征数n_features(即特征向量)
    n_samples, n_features = X.shape
    #对矩阵进行零均值化，即减去这一行的均值
    mean = np.array([np.mean(X[:,i]) for i in range(n_features)])
    norm_X = X - mean

    #散度矩阵scatter_matrix
    #散度矩阵就是协方差矩阵*(总数据量-1),因此他们的特征根和特征向量是一样的
    scatter_matrix = np.dot(np.transpose(norm_X),norm_X)

    #计算特征向量和特征值
    eig_val, eig_vec = np.linalg.eig(scatter_matrix)
    eig_pairs = [(np.abs(eig_val[i]), eig_vec[:,i]) for i in range(n_features)]

    #根据特征值大小，从大到小对(特征值,特征向量)的多对pair进行排序
    eig_pairs.sort(reverse=True)
    #选出前k个特征向量
    feature=np.array([ele[1] for ele in eig_pairs[:k]])
    
    #输出降维后的特征向量
    print("降维后的特征向量为：\n")
    for k in range(16):
        for i in range(12):
            for j in range(12):
                print("%.3f  "%(feature[k][i*12+j]),end="")
            print("\n")
        print("\n")
    print("-----------------------------------------------------------------------------")

    #将特征向量rescale到0——255之间
    array = feature
    ymax = 255
    ymin = 0
    xmax = max(map(max,array))
    xmin = min(map(min,array))
    for i in range(16):
        for j in range(144):
            array[i][j] = int(round(((ymax-ymin)*(array[i][j]-xmin)/(xmax-xmin))+ymin))
    print("rescale到0—255范围内的特征向量为：\n")
    for k in range(16):
        for i in range(12):
            for j in range(12):
                print("%d  "%(array[k][i*12+j]),end="")
            print("\n")
        print("\n")
    print("-----------------------------------------------------------------------------")
    feature = np.array([ele[1] for ele in eig_pairs[:temp]])
    
    #Y = PX即为降维到k维后的数据
    data = np.dot(norm_X,np.transpose(feature))
    return data


#存储裁剪后图片的每个子图(12*12)
X = np.zeros((12,12),dtype = int)
#存储所有子图，组成训练集
X_new = np.zeros((144,144),dtype = int)

#读取代表图片
#img_num可修改为自己需要读取的图片编号
img_number = "915"
img = cv2.imread("C:/Users/93508/Desktop/Final/data/imgs/" + img_number + ".png")
#以灰度图的方式读取，去除图片的三通道RGB信息
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#将图片尺寸裁剪为12的倍数
img = cv2.resize(img,(144,144))

#将图片拆分12*12的子图并全部保存至X_new
#每个12*12的子图被视为144维的向量，将其一维扁平化存入X_new的每一行中
#由于图片为144*144，子图为12*12，所以X_new为144*144
#即144条数据，每条数据有144维的特征
for i in range(0,144):
    X = img[int(i/12)*12:int(i/12)*12+12,i%12*12:i%12*12+12]
    X_new[i,:] = X.reshape(1,-1)

#调用手工实现的PCA
result = pca(X_new,16)
print("压缩结果为：\n")
print(result)


降维后的特征向量为：

0.079  0.082  0.082  0.082  0.085  0.085  0.084  0.084  0.078  0.081  0.076  0.077  

0.066  0.068  0.069  0.071  0.073  0.072  0.073  0.074  0.071  0.073  0.068  0.071  

0.054  0.055  0.054  0.055  0.056  0.057  0.058  0.056  0.055  0.057  0.056  0.057  

0.072  0.070  0.066  0.068  0.070  0.070  0.071  0.070  0.066  0.069  0.068  0.067  

0.094  0.093  0.090  0.088  0.087  0.090  0.091  0.090  0.092  0.096  0.091  0.088  

0.102  0.102  0.100  0.100  0.098  0.098  0.099  0.100  0.103  0.105  0.100  0.096  

0.104  0.104  0.102  0.101  0.102  0.099  0.102  0.105  0.105  0.105  0.100  0.098  

0.089  0.089  0.088  0.089  0.091  0.089  0.090  0.091  0.093  0.092  0.088  0.087  

0.068  0.068  0.067  0.070  0.072  0.070  0.072  0.072  0.073  0.070  0.068  0.065  

0.076  0.074  0.073  0.073  0.075  0.075  0.076  0.078  0.076  0.075  0.073  0.073  

0.091  0.093  0.090  0.089  0.093  0.092  0.095  0.096  0.091  0.088  0.086  0.087  

0.090  0.094  0.091  0.091  0.093  0.095  

0.047  0.016  -0.028  -0.059  -0.073  -0.131  -0.075  -0.015  0.024  0.028  -0.011  -0.038  

0.096  0.124  0.067  0.022  0.014  -0.029  -0.073  -0.050  -0.030  -0.047  -0.071  -0.107  

0.092  0.116  0.113  0.098  0.079  0.041  0.007  0.036  0.058  0.077  0.041  -0.018  

-0.031  0.005  -0.009  -0.066  -0.092  -0.065  -0.035  0.044  0.098  0.182  0.175  0.083  

-0.115  -0.121  -0.116  -0.158  -0.175  -0.124  -0.062  0.056  0.112  0.110  0.134  0.090  

-0.041  -0.038  -0.048  -0.068  -0.059  0.001  0.028  0.119  0.141  0.068  0.066  0.036  

0.007  0.037  0.069  0.063  0.066  0.072  0.084  0.097  0.111  -0.000  -0.022  -0.034  

-0.002  0.024  0.069  0.060  0.064  0.044  0.025  -0.011  -0.053  -0.193  -0.204  -0.206  

-0.044  -0.009  0.083  0.095  0.103  0.066  -0.008  -0.033  -0.099  -0.207  -0.215  -0.227  

-0.134  -0.086  0.019  0.037  0.024  0.013  -0.053  -0.062  -0.043  -0.071  -0.073  -0.108  

-0.094  -0.057  0.013  0.029  -0.015  -0.014  -0.046  -0.046  0.024  0.030  0.047

115  107  147  171  185  168  179  200  217  148  115  106  

103  87  101  135  182  181  155  178  222  191  189  191  

54  36  37  84  119  112  83  112  128  161  181  179  

120  112  117  151  150  102  76  85  84  138  164  147  

168  169  178  193  191  161  123  115  96  139  151  134  



129  122  98  110  125  114  107  99  139  161  207  190  

153  161  109  130  152  139  120  100  145  182  210  177  

202  200  165  179  191  136  137  102  120  162  157  114  

150  171  169  156  162  122  125  108  131  120  55  24  

77  88  124  136  146  128  110  121  155  122  52  31  

50  57  93  123  186  178  146  180  201  144  110  105  

52  65  91  143  206  215  157  208  237  171  155  166  

44  58  76  117  158  145  155  202  221  170  157  159  

107  116  122  129  124  75  98  152  182  158  130  97  

181  186  206  161  104  84  100  142  185  154  102  65  

193  203  216  172  111  80  98  145  184  208  170  135  

179  177  186  161  97  34  44  67  94  

问题1——调用Sklearn库的PCA函数，特征向量降为16D

In [28]:
import numpy as np
import os
import cv2
import shutil
import math
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")


#存储裁剪后图片的每个子图(12*12)
X = np.zeros((12,12),dtype = int)
#存储所有子图，组成训练集
X_new = np.zeros((144,144),dtype = int)

#读取代表图片
#img_num可修改为自己需要读取的图片编号
img_number = "915"
img = cv2.imread("C:/Users/93508/Desktop/Final/data/imgs/" + img_number + ".png")
#以灰度图的方式读取，去除图片的三通道RGB信息
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#将图片尺寸裁剪为12的倍数
img = cv2.resize(img,(144,144))

#将图片拆分12*12的子图并全部保存至X_new
#每个12*12的子图被视为144维的向量，将其一维扁平化存入X_new的每一行中
#由于图片为144*144，子图为12*12，所以X_new为144*144
#即144条数据，每条数据有144维的特征
for i in range(0,144):
    X = img[int(i/12)*12:int(i/12)*12+12,i%12*12:i%12*12+12]
    X_new[i,:] = X.reshape(1,-1)


# 可修改n的数量来选择需要保留的特征数量（即降维后的结果）
n = 16
#调用sklearn库中的PCA
#n_components: 需要保留的特征数量（即降维后的结果）
pca = PCA(n_components = n) #实例化
newX = pca.fit_transform(X_new) #用已有数据训练PCA模型，并返回降维后的数据

#PCA中的属性components_为降维后的特征向量
print("降维后的特征向量为：\n")
for k in range(16):
    for i in range(12):
        for j in range(12):
            print("%.3f  "%(pca.components_[k][i*12+j]),end="")
        print("\n")
    print("\n")
print("-----------------------------------------------------------------------------")

#将特征向量rescale到0——255之间
array = pca.components_
ymax = 255
ymin = 0
xmax = max(map(max,array))
xmin = min(map(min,array))
for i in range(16):
    for j in range(144):
        array[i][j] = int(round(((ymax-ymin)*(array[i][j]-xmin)/(xmax-xmin))+ymin))
print("rescale到0—255范围内的特征向量为：\n")
for k in range(16):
    for i in range(12):
        for j in range(12):
            print("%d  "%(array[k][i*12+j]),end="")
        print("\n")
    print("\n")
print("-----------------------------------------------------------------------------")

print("压缩结果为：\n")
print(newX)


降维后的特征向量为：

0.079  0.082  0.082  0.082  0.085  0.085  0.084  0.084  0.078  0.081  0.076  0.077  

0.066  0.068  0.069  0.071  0.073  0.072  0.073  0.074  0.071  0.073  0.068  0.071  

0.054  0.055  0.054  0.055  0.056  0.057  0.058  0.056  0.055  0.057  0.056  0.057  

0.072  0.070  0.066  0.068  0.070  0.070  0.071  0.070  0.066  0.069  0.068  0.067  

0.094  0.093  0.090  0.088  0.087  0.090  0.091  0.090  0.092  0.096  0.091  0.088  

0.102  0.102  0.100  0.100  0.098  0.098  0.099  0.100  0.103  0.105  0.100  0.096  

0.104  0.104  0.102  0.101  0.102  0.099  0.102  0.105  0.105  0.105  0.100  0.098  

0.089  0.089  0.088  0.089  0.091  0.089  0.090  0.091  0.093  0.092  0.088  0.087  

0.068  0.068  0.067  0.070  0.072  0.070  0.072  0.072  0.073  0.070  0.068  0.065  

0.076  0.074  0.073  0.073  0.075  0.075  0.076  0.078  0.076  0.075  0.073  0.073  

0.091  0.093  0.090  0.089  0.093  0.092  0.095  0.096  0.091  0.088  0.086  0.087  

0.090  0.094  0.091  0.091  0.093  0.095  

-0.065  -0.074  -0.050  -0.049  -0.046  -0.014  -0.019  0.041  0.072  0.093  0.098  0.049  

-0.012  0.009  -0.031  -0.071  -0.072  -0.011  -0.002  0.020  0.030  0.026  0.035  -0.016  

0.044  0.023  0.003  0.014  -0.028  0.028  0.034  0.014  0.006  -0.028  -0.053  -0.142  

0.071  0.077  0.058  0.015  -0.029  0.016  0.045  0.027  -0.003  -0.059  -0.121  -0.213  

0.091  0.054  0.044  0.018  -0.040  0.013  0.025  0.015  -0.014  -0.099  -0.131  -0.218  

0.131  0.072  0.043  0.009  -0.023  0.018  -0.013  -0.027  -0.071  -0.149  -0.127  -0.212  

0.152  0.101  0.070  0.027  -0.000  0.043  0.001  -0.009  -0.053  -0.136  -0.126  -0.167  

0.157  0.120  0.112  0.055  0.008  0.032  0.021  0.024  0.011  -0.037  -0.008  -0.038  

0.037  0.018  0.005  -0.093  -0.136  -0.076  -0.061  -0.037  0.063  0.070  0.062  -0.004  

0.028  0.017  -0.027  -0.141  -0.129  -0.091  -0.052  0.016  0.148  0.145  0.120  0.036  

-0.030  -0.010  -0.054  -0.127  -0.116  -0.083  -0.017  0.088  0.172  0.169  0.119  0

89  78  79  91  102  157  172  173  158  152  153  141  

48  38  25  32  42  102  109  112  109  104  92  90  

72  60  58  61  59  110  109  110  106  86  82  81  

106  100  97  98  94  132  130  128  125  98  89  87  

137  143  146  144  139  168  176  166  153  119  112  102  

154  165  172  177  177  205  218  209  195  166  158  143  

109  120  128  141  151  180  192  200  185  162  156  133  

69  71  79  90  94  112  115  128  95  79  69  66  

135  135  141  144  136  158  159  151  107  85  72  73  

171  168  171  175  168  188  184  168  131  99  92  88  



224  211  208  198  156  171  166  136  84  81  68  64  

226  226  207  183  168  176  168  130  71  64  54  51  

214  209  189  182  171  171  163  128  89  78  65  68  

177  169  153  137  137  146  142  127  107  98  86  87  

151  145  131  118  118  147  138  132  113  107  96  88  

120  110  105  102  105  144  135  138  114  106  98  87  

105  101  90  89  105  146  155  163  141  128  121  116  

100  

问题2——144D

In [26]:
import numpy as np
import os
import cv2
import shutil
import math
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

#手工实现PCA
# X代表训练集矩阵
# k代表需要保留的特征数量（即降维后的特征向量数量）
def pca(X,k):
    temp = k
    #矩阵行数代表样本数量n_samples，矩阵列数代表每个样本的特征数n_features(即特征向量)
    n_samples, n_features = X.shape
    #对矩阵进行零均值化，即减去这一行的均值
    mean = np.array([np.mean(X[:,i]) for i in range(n_features)])
    norm_X = X - mean

    #散度矩阵scatter_matrix
    #散度矩阵就是协方差矩阵*(总数据量-1),因此他们的特征根和特征向量是一样的
    scatter_matrix = np.dot(np.transpose(norm_X),norm_X)

    #计算特征向量和特征值
    eig_val, eig_vec = np.linalg.eig(scatter_matrix)
    eig_pairs = [(np.abs(eig_val[i]), eig_vec[:,i]) for i in range(n_features)]

    #根据特征值大小，从大到小对(特征值,特征向量)的多对pair进行排序
    eig_pairs.sort(reverse=True)
    #选出前k个特征向量
    feature=np.array([ele[1] for ele in eig_pairs[:k]])
    
    #输出降维后的特征向量
    print("降维后的特征向量为：\n")
    for k in range(144):
        for i in range(12):
            for j in range(12):
                print("%.3f  "%(feature[k][i*12+j]),end="")
            print("\n")
        print("\n")
    print("-----------------------------------------------------------------------------")

    #将特征向量rescale到0——255之间
    array = feature
    ymax = 255
    ymin = 0
    xmax = max(map(max,array))
    xmin = min(map(min,array))
    for i in range(144):
        for j in range(144):
            array[i][j] = int(round(((ymax-ymin)*(array[i][j]-xmin)/(xmax-xmin))+ymin))
    print("rescale到0—255范围内的特征向量为：\n")
    for k in range(144):
        for i in range(12):
            for j in range(12):
                print("%d  "%(array[k][i*12+j]),end="")
            print("\n")
        print("\n")
    print("-----------------------------------------------------------------------------")
    feature = np.array([ele[1] for ele in eig_pairs[:temp]])
    
    #Y = PX即为降维到k维后的数据
    data = np.dot(norm_X,np.transpose(feature))
    return data


#存储裁剪后图片的每个子图(12*12)
X = np.zeros((12,12),dtype = int)
#存储所有子图，组成训练集
X_new = np.zeros((144,144),dtype = int)

#读取代表图片
#img_num可修改为自己需要读取的图片编号
img_number = "915"
img = cv2.imread("C:/Users/93508/Desktop/Final/data/imgs/" + img_number + ".png")
#以灰度图的方式读取，去除图片的三通道RGB信息
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#将图片尺寸裁剪为12的倍数
img = cv2.resize(img,(144,144))

#将图片拆分12*12的子图并全部保存至X_new
#每个12*12的子图被视为144维的向量，将其一维扁平化存入X_new的每一行中
#由于图片为144*144，子图为12*12，所以X_new为144*144
#即144条数据，每条数据有144维的特征
for i in range(0,144):
    X = img[int(i/12)*12:int(i/12)*12+12,i%12*12:i%12*12+12]
    X_new[i,:] = X.reshape(1,-1)

#调用手工实现的PCA
result = pca(X_new,144)
print("压缩结果为：\n")
print(result)


降维后的特征向量为：

0.079  0.082  0.082  0.082  0.085  0.085  0.084  0.084  0.078  0.081  0.076  0.077  

0.066  0.068  0.069  0.071  0.073  0.072  0.073  0.074  0.071  0.073  0.068  0.071  

0.054  0.055  0.054  0.055  0.056  0.057  0.058  0.056  0.055  0.057  0.056  0.057  

0.072  0.070  0.066  0.068  0.070  0.070  0.071  0.070  0.066  0.069  0.068  0.067  

0.094  0.093  0.090  0.088  0.087  0.090  0.091  0.090  0.092  0.096  0.091  0.088  

0.102  0.102  0.100  0.100  0.098  0.098  0.099  0.100  0.103  0.105  0.100  0.096  

0.104  0.104  0.102  0.101  0.102  0.099  0.102  0.105  0.105  0.105  0.100  0.098  

0.089  0.089  0.088  0.089  0.091  0.089  0.090  0.091  0.093  0.092  0.088  0.087  

0.068  0.068  0.067  0.070  0.072  0.070  0.072  0.072  0.073  0.070  0.068  0.065  

0.076  0.074  0.073  0.073  0.075  0.075  0.076  0.078  0.076  0.075  0.073  0.073  

0.091  0.093  0.090  0.089  0.093  0.092  0.095  0.096  0.091  0.088  0.086  0.087  

0.090  0.094  0.091  0.091  0.093  0.095  

-0.044  -0.023  -0.003  -0.014  0.028  -0.028  -0.034  -0.014  -0.006  0.028  0.053  0.142  

-0.071  -0.077  -0.058  -0.015  0.029  -0.016  -0.045  -0.027  0.003  0.059  0.121  0.213  

-0.091  -0.054  -0.044  -0.018  0.040  -0.013  -0.025  -0.015  0.014  0.099  0.131  0.218  

-0.131  -0.072  -0.043  -0.009  0.023  -0.018  0.013  0.027  0.071  0.149  0.127  0.212  

-0.152  -0.101  -0.070  -0.027  0.000  -0.043  -0.001  0.009  0.053  0.136  0.126  0.167  

-0.157  -0.120  -0.112  -0.055  -0.008  -0.032  -0.021  -0.024  -0.011  0.037  0.008  0.038  

-0.037  -0.018  -0.005  0.093  0.136  0.076  0.061  0.037  -0.063  -0.070  -0.062  0.004  

-0.028  -0.017  0.027  0.141  0.129  0.091  0.052  -0.016  -0.148  -0.145  -0.120  -0.036  

0.030  0.010  0.054  0.127  0.116  0.083  0.017  -0.088  -0.172  -0.169  -0.119  -0.065  



0.047  0.016  -0.028  -0.059  -0.073  -0.131  -0.075  -0.015  0.024  0.028  -0.011  -0.038  

0.096  0.124  0.067  0.022  0.014  -0.029  -0.073  -0.050  -0.030  -0.

0.018  -0.124  -0.144  -0.073  -0.029  -0.066  -0.094  -0.100  -0.082  -0.012  -0.009  0.001  

0.063  -0.028  -0.044  0.040  0.064  -0.010  -0.094  -0.139  -0.131  -0.077  -0.058  -0.049  

0.074  -0.011  -0.036  0.036  0.111  0.109  0.054  0.014  0.024  0.072  0.142  0.113  

-0.046  -0.125  -0.162  -0.060  -0.008  0.008  0.001  -0.003  -0.007  0.013  0.084  0.093  

0.060  0.046  0.005  0.066  0.021  0.003  -0.030  -0.025  0.088  0.026  0.005  0.004  

0.053  0.059  0.016  0.035  -0.001  -0.002  -0.029  -0.040  0.013  -0.053  -0.117  -0.094  



0.102  0.111  0.093  0.039  -0.020  -0.014  -0.055  -0.067  -0.035  -0.050  -0.218  -0.141  

-0.019  0.020  0.080  0.056  -0.007  -0.020  0.006  0.019  0.078  0.098  0.031  0.117  

-0.190  -0.069  0.005  0.005  -0.044  -0.047  -0.082  -0.047  -0.018  -0.016  0.017  0.121  

-0.197  -0.085  0.030  0.091  0.124  0.017  -0.071  -0.029  -0.043  -0.064  0.030  0.101  

-0.145  -0.022  0.071  0.125  0.162  0.094  -0.034  -0.034  0.021  0.069  0.

-0.053  -0.000  0.021  -0.084  0.004  0.019  0.086  0.093  0.087  0.062  -0.018  0.035  

-0.077  0.014  0.047  -0.034  0.097  0.065  0.132  0.013  -0.008  0.032  -0.017  -0.051  

-0.082  0.011  -0.014  -0.129  0.090  -0.021  -0.004  -0.053  -0.070  0.011  -0.022  -0.003  

-0.081  0.023  0.028  -0.059  0.095  -0.030  -0.009  -0.086  -0.062  0.069  0.090  0.089  



0.130  0.108  0.092  -0.092  -0.067  0.022  0.055  0.040  0.002  -0.125  -0.003  -0.046  

-0.009  0.013  -0.005  -0.102  -0.169  -0.011  0.036  0.036  0.071  -0.026  0.073  -0.006  

0.016  -0.066  0.047  0.031  -0.131  -0.062  -0.021  0.002  -0.025  -0.010  0.070  0.010  

-0.026  -0.045  0.120  0.170  -0.061  -0.003  0.019  0.071  0.015  -0.070  -0.033  -0.080  

-0.013  -0.100  0.015  0.084  0.001  -0.049  -0.045  -0.043  -0.094  -0.081  -0.114  -0.093  

0.108  0.045  0.048  0.012  0.002  0.050  -0.020  0.000  0.017  -0.006  -0.020  0.092  

0.095  0.088  0.041  -0.052  -0.082  0.137  0.146  0.048  0.020  0.057  0.065

-0.107  0.029  -0.141  -0.060  -0.019  -0.092  -0.000  0.130  -0.161  -0.046  -0.167  0.018  

-0.042  -0.043  0.054  0.105  0.131  -0.017  -0.025  0.084  0.059  0.120  0.024  0.270  

-0.001  -0.117  0.037  -0.195  -0.124  -0.078  -0.132  -0.076  -0.048  0.026  -0.077  0.021  

0.183  0.005  0.107  0.021  0.043  0.189  0.082  -0.010  0.063  0.130  -0.067  -0.137  

0.015  -0.068  0.030  0.011  -0.065  0.046  0.055  0.044  -0.093  0.100  -0.021  -0.113  

-0.075  0.021  -0.000  -0.047  -0.042  -0.074  -0.002  0.074  -0.074  0.100  0.059  0.037  

-0.080  0.015  0.072  0.037  0.002  0.020  -0.013  0.037  -0.063  0.020  0.006  -0.010  

-0.040  0.039  0.093  0.086  0.069  -0.013  -0.080  -0.011  -0.051  -0.017  -0.055  -0.011  

-0.002  -0.118  -0.051  0.012  0.028  -0.007  -0.021  -0.006  -0.068  -0.020  0.019  0.007  

0.135  -0.121  -0.043  0.121  -0.057  -0.021  0.142  0.148  -0.003  -0.050  0.062  0.044  

0.247  -0.049  -0.057  0.076  -0.198  -0.214  -0.034  0.031  0.079  -0.086  -

...

146  143  151  55  125  134  143  152  139  138  40  186  

150  125  187  111  101  121  172  189  114  163  75  176  

136  99  146  116  106  90  106  84  139  121  88  151  

168  124  106  177  165  129  193  104  193  162  151  171  

134  132  102  150  59  70  142  60  126  143  121  109  

104  157  117  171  170  191  193  102  170  127  115  92  

105  147  101  97  164  144  146  71  130  113  121  109  

91  112  139  151  178  99  64  119  170  141  188  189  

146  192  159  99  125  137  174  134  110  101  104  53  

120  107  87  117  94  130  165  145  129  113  172  174  

113  159  145  159  125  106  160  138  160  120  139  106  

150  114  95  185  174  99  106  99  155  76  180  146  



96  179  124  137  135  104  147  108  215  115  109  111  

85  107  116  147  143  133  158  98  106  155  147  157  

123  123  184  112  151  157  129  158  49  140  118  126  

112  123  164  108  208  106  116  188  127  191  134  75  

138  128  91  72  125  122  79  128

93  122  127  209  143  89  135  98  131  195  104  68  

121  125  103  101  77  184  169  109  152  117  122  179  

141  135  141  161  176  144  95  125  137  115  107  155  

107  126  151  113  162  105  107  178  110  136  188  69  



153  125  92  97  159  174  98  147  115  148  129  140  

154  171  133  143  173  115  63  100  167  101  128  121  

87  80  133  114  118  146  161  151  161  159  94  184  

135  138  189  164  129  147  105  186  22  127  143  105  

198  122  160  73  80  146  121  170  114  177  85  138  

128  62  123  124  173  104  129  168  151  167  118  177  

119  149  103  125  192  108  151  69  121  95  121  107  

103  182  130  132  114  104  192  87  190  74  127  197  

133  170  122  159  42  114  163  98  172  163  136  83  

146  67  120  188  106  161  141  148  109  118  142  111  

159  114  131  127  87  129  127  106  148  151  131  152  

103  174  109  91  234  135  115  145  141  119  113  132  



149  158  114  164  154  101  101

103  106  169  175  86  159  154  116  132  104  121  126  

180  142  58  169  144  92  119  105  125  177  149  130  

102  114  180  110  154  152  101  159  143  83  173  110  

139  165  116  104  159  87  209  83  179  58  150  120  



107  132  184  138  112  117  123  159  105  143  126  126  

131  134  137  49  159  156  140  110  144  102  197  126  

138  119  145  156  126  104  146  119  228  47  72  134  

133  140  77  129  140  147  113  123  135  177  159  156  

118  129  114  132  125  193  127  121  100  111  123  119  

151  187  153  185  114  96  89  156  136  170  131  95  

117  36  100  97  213  156  165  139  100  116  186  108  

146  155  174  67  65  163  135  91  184  75  160  160  

128  107  162  152  143  189  97  101  119  152  94  159  

107  156  133  94  167  55  156  170  135  135  110  121  

119  99  157  183  143  64  193  73  160  104  150  157  

158  144  128  102  124  113  135  187  83  147  153  87  



146  150  105  148  94  161  108 

80  166  158  112  125  145  86  185  191  100  99  117  

176  59  134  166  88  155  142  132  79  113  189  137  

122  152  175  76  177  97  123  131  161  123  98  154  



121  130  114  137  98  207  76  146  122  124  139  152  

140  124  197  104  170  39  168  159  137  121  163  59  

139  135  98  129  134  139  158  117  124  103  124  183  

141  114  106  159  152  126  66  112  196  139  142  124  

121  123  124  155  111  151  176  166  130  112  159  90  

143  142  111  90  170  89  111  104  110  186  117  163  

106  146  186  116  137  129  182  133  58  102  113  165  

154  57  174  101  128  158  76  160  233  104  142  104  

139  164  78  177  129  108  147  126  87  88  169  140  

112  122  88  204  93  162  156  98  139  196  68  125  

152  143  151  61  168  144  103  194  129  107  146  118  

131  118  182  103  130  96  164  72  120  167  127  140  



114  100  193  117  124  120  122  165  91  137  125  157  

146  144  103  125  181  84  223  62

151  107  122  161  130  118  105  194  86  114  164  123  

130  147  123  134  118  156  117  105  169  120  126  136  



135  141  133  85  155  145  129  83  164  162  99  145  

123  133  147  123  140  129  137  175  105  59  165  130  

121  134  110  162  170  71  139  86  152  196  143  109  

150  146  120  130  118  122  163  140  119  82  158  115  

116  133  99  207  54  197  107  167  85  184  131  112  

135  115  138  77  205  71  129  105  228  57  138  164  

132  139  135  175  98  152  118  179  43  176  93  148  

124  137  161  112  108  126  124  123  133  198  117  102  

114  185  88  138  112  183  122  150  94  130  90  174  

167  81  155  100  175  114  64  154  152  129  195  78  

107  182  102  126  117  133  187  86  150  146  93  137  

135  97  170  117  150  105  156  159  112  117  133  136  



126  93  165  120  148  124  130  129  117  138  117  163  

168  147  62  194  91  123  160  118  138  117  175  81  

106  174  155  71  133  208  62  1

问题2——60D

In [24]:
import numpy as np
import os
import cv2
import shutil
import math
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

#手工实现PCA
# X代表训练集矩阵
# k代表需要保留的特征数量（即降维后的特征向量数量）
def pca(X,k):
    temp = k
    #矩阵行数代表样本数量n_samples，矩阵列数代表每个样本的特征数n_features(即特征向量)
    n_samples, n_features = X.shape
    #对矩阵进行零均值化，即减去这一行的均值
    mean = np.array([np.mean(X[:,i]) for i in range(n_features)])
    norm_X = X - mean

    #散度矩阵scatter_matrix
    #散度矩阵就是协方差矩阵*(总数据量-1),因此他们的特征根和特征向量是一样的
    scatter_matrix = np.dot(np.transpose(norm_X),norm_X)

    #计算特征向量和特征值
    eig_val, eig_vec = np.linalg.eig(scatter_matrix)
    eig_pairs = [(np.abs(eig_val[i]), eig_vec[:,i]) for i in range(n_features)]

    #根据特征值大小，从大到小对(特征值,特征向量)的多对pair进行排序
    eig_pairs.sort(reverse=True)
    #选出前k个特征向量
    feature=np.array([ele[1] for ele in eig_pairs[:k]])
    
    #输出降维后的特征向量
    print("降维后的特征向量为：\n")
    for k in range(60):
        for i in range(12):
            for j in range(12):
                print("%.3f  "%(feature[k][i*12+j]),end="")
            print("\n")
        print("\n")
    print("-----------------------------------------------------------------------------")

    #将特征向量rescale到0——255之间
    array = feature
    ymax = 255
    ymin = 0
    xmax = max(map(max,array))
    xmin = min(map(min,array))
    for i in range(60):
        for j in range(144):
            array[i][j] = int(round(((ymax-ymin)*(array[i][j]-xmin)/(xmax-xmin))+ymin))
    print("rescale到0—255范围内的特征向量为：\n")
    for k in range(60):
        for i in range(12):
            for j in range(12):
                print("%d  "%(array[k][i*12+j]),end="")
            print("\n")
        print("\n")
    print("-----------------------------------------------------------------------------")
    feature = np.array([ele[1] for ele in eig_pairs[:temp]])
    
    #Y = PX即为降维到k维后的数据
    data = np.dot(norm_X,np.transpose(feature))
    return data


#存储裁剪后图片的每个子图(12*12)
X = np.zeros((12,12),dtype = int)
#存储所有子图，组成训练集
X_new = np.zeros((144,144),dtype = int)

#读取代表图片
#img_num可修改为自己需要读取的图片编号
img_number = "915"
img = cv2.imread("C:/Users/93508/Desktop/Final/data/imgs/" + img_number + ".png")
#以灰度图的方式读取，去除图片的三通道RGB信息
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#将图片尺寸裁剪为12的倍数
img = cv2.resize(img,(144,144))

#将图片拆分12*12的子图并全部保存至X_new
#每个12*12的子图被视为144维的向量，将其一维扁平化存入X_new的每一行中
#由于图片为144*144，子图为12*12，所以X_new为144*144
#即144条数据，每条数据有144维的特征
for i in range(0,144):
    X = img[int(i/12)*12:int(i/12)*12+12,i%12*12:i%12*12+12]
    X_new[i,:] = X.reshape(1,-1)

#调用手工实现的PCA
result = pca(X_new,60)
print("压缩结果为：\n")
print(result)


降维后的特征向量为：

0.079  0.082  0.082  0.082  0.085  0.085  0.084  0.084  0.078  0.081  0.076  0.077  

0.066  0.068  0.069  0.071  0.073  0.072  0.073  0.074  0.071  0.073  0.068  0.071  

0.054  0.055  0.054  0.055  0.056  0.057  0.058  0.056  0.055  0.057  0.056  0.057  

0.072  0.070  0.066  0.068  0.070  0.070  0.071  0.070  0.066  0.069  0.068  0.067  

0.094  0.093  0.090  0.088  0.087  0.090  0.091  0.090  0.092  0.096  0.091  0.088  

0.102  0.102  0.100  0.100  0.098  0.098  0.099  0.100  0.103  0.105  0.100  0.096  

0.104  0.104  0.102  0.101  0.102  0.099  0.102  0.105  0.105  0.105  0.100  0.098  

0.089  0.089  0.088  0.089  0.091  0.089  0.090  0.091  0.093  0.092  0.088  0.087  

0.068  0.068  0.067  0.070  0.072  0.070  0.072  0.072  0.073  0.070  0.068  0.065  

0.076  0.074  0.073  0.073  0.075  0.075  0.076  0.078  0.076  0.075  0.073  0.073  

0.091  0.093  0.090  0.089  0.093  0.092  0.095  0.096  0.091  0.088  0.086  0.087  

0.090  0.094  0.091  0.091  0.093  0.095  

0.030  0.010  0.054  0.127  0.116  0.083  0.017  -0.088  -0.172  -0.169  -0.119  -0.065  



0.047  0.016  -0.028  -0.059  -0.073  -0.131  -0.075  -0.015  0.024  0.028  -0.011  -0.038  

0.096  0.124  0.067  0.022  0.014  -0.029  -0.073  -0.050  -0.030  -0.047  -0.071  -0.107  

0.092  0.116  0.113  0.098  0.079  0.041  0.007  0.036  0.058  0.077  0.041  -0.018  

-0.031  0.005  -0.009  -0.066  -0.092  -0.065  -0.035  0.044  0.098  0.182  0.175  0.083  

-0.115  -0.121  -0.116  -0.158  -0.175  -0.124  -0.062  0.056  0.112  0.110  0.134  0.090  

-0.041  -0.038  -0.048  -0.068  -0.059  0.001  0.028  0.119  0.141  0.068  0.066  0.036  

0.007  0.037  0.069  0.063  0.066  0.072  0.084  0.097  0.111  -0.000  -0.022  -0.034  

-0.002  0.024  0.069  0.060  0.064  0.044  0.025  -0.011  -0.053  -0.193  -0.204  -0.206  

-0.044  -0.009  0.083  0.095  0.103  0.066  -0.008  -0.033  -0.099  -0.207  -0.215  -0.227  

-0.134  -0.086  0.019  0.037  0.024  0.013  -0.053  -0.062  -0.043  -0.071  -0.073

0.155  0.149  0.051  0.011  -0.012  0.053  0.065  0.097  0.072  0.035  -0.060  -0.119  

0.117  0.112  0.020  -0.018  -0.057  -0.006  -0.036  -0.084  -0.111  -0.075  -0.128  -0.254  

0.109  0.027  0.040  0.013  0.015  0.037  -0.000  -0.014  0.038  0.093  0.053  -0.061  

-0.011  -0.106  -0.089  -0.116  -0.099  -0.023  -0.048  -0.011  0.081  0.149  0.098  0.020  



-0.064  -0.018  0.077  0.052  0.046  0.082  -0.038  -0.166  -0.144  0.099  0.096  0.129  

-0.053  -0.020  0.061  -0.008  -0.015  0.054  -0.039  -0.162  -0.150  0.001  0.046  0.092  

-0.086  -0.009  0.066  -0.001  -0.049  0.130  0.100  -0.010  0.037  -0.003  0.007  -0.049  

-0.154  -0.053  -0.009  0.023  -0.012  0.078  0.106  0.065  0.075  0.022  -0.122  -0.213  

-0.154  -0.077  -0.065  0.007  0.006  -0.043  -0.030  -0.042  0.015  0.093  -0.097  -0.158  

-0.009  0.090  0.061  0.042  0.014  -0.074  -0.088  -0.089  0.070  0.172  -0.007  -0.052  

0.108  0.106  0.055  0.040  0.024  -0.013  -0.080  -0.047  0.052  0.135  0.0

-0.077  0.014  0.047  -0.034  0.097  0.065  0.132  0.013  -0.008  0.032  -0.017  -0.051  

-0.082  0.011  -0.014  -0.129  0.090  -0.021  -0.004  -0.053  -0.070  0.011  -0.022  -0.003  

-0.081  0.023  0.028  -0.059  0.095  -0.030  -0.009  -0.086  -0.062  0.069  0.090  0.089  



0.130  0.108  0.092  -0.092  -0.067  0.022  0.055  0.040  0.002  -0.125  -0.003  -0.046  

-0.009  0.013  -0.005  -0.102  -0.169  -0.011  0.036  0.036  0.071  -0.026  0.073  -0.006  

0.016  -0.066  0.047  0.031  -0.131  -0.062  -0.021  0.002  -0.025  -0.010  0.070  0.010  

-0.026  -0.045  0.120  0.170  -0.061  -0.003  0.019  0.071  0.015  -0.070  -0.033  -0.080  

-0.013  -0.100  0.015  0.084  0.001  -0.049  -0.045  -0.043  -0.094  -0.081  -0.114  -0.093  

0.108  0.045  0.048  0.012  0.002  0.050  -0.020  0.000  0.017  -0.006  -0.020  0.092  

0.095  0.088  0.041  -0.052  -0.082  0.137  0.146  0.048  0.020  0.057  0.065  0.147  

-0.057  -0.069  -0.142  -0.145  -0.205  -0.070  0.067  -0.001  -0.094  -0.067  

-0.033  -0.032  -0.004  0.018  -0.099  0.047  0.181  0.110  0.091  -0.105  -0.109  -0.078  

-0.021  0.019  0.056  -0.126  -0.198  -0.010  0.094  -0.002  0.108  -0.040  0.005  -0.021  

0.045  0.060  0.178  0.013  -0.125  -0.008  -0.013  -0.075  0.063  0.060  -0.023  0.013  

-0.054  0.053  0.115  0.120  -0.046  -0.035  -0.015  -0.046  0.036  0.011  -0.105  -0.020  

-0.005  -0.118  -0.043  0.069  -0.041  -0.048  0.023  -0.051  -0.078  0.195  0.050  -0.023  

0.099  -0.096  -0.099  0.036  -0.000  0.009  0.092  -0.007  -0.061  0.217  0.037  -0.018  

0.070  -0.135  -0.042  0.080  0.033  -0.053  0.117  -0.023  -0.217  0.051  -0.121  -0.130  

0.130  -0.030  -0.036  0.093  0.123  -0.035  0.114  -0.003  -0.072  0.095  -0.096  0.018  

-0.045  -0.161  -0.099  0.071  0.101  -0.179  -0.123  -0.029  0.124  -0.007  -0.075  0.104  

-0.007  -0.099  -0.017  0.014  0.087  -0.074  -0.051  0.138  0.086  -0.075  -0.107  0.089  

0.103  0.107  0.044  0.035  0.058  0.023  0.052  -0.044  -0.057  0.045  

-0.021  0.009  -0.054  0.045  0.131  -0.033  -0.085  0.101  0.042  -0.012  -0.100  -0.067  

0.010  0.019  -0.016  0.047  -0.067  -0.014  -0.083  0.018  -0.003  0.061  0.004  0.014  

0.137  -0.037  -0.111  -0.086  -0.077  0.067  0.044  -0.044  -0.032  0.124  -0.187  -0.001  

-0.076  0.007  0.007  0.103  0.087  0.062  0.053  -0.140  -0.016  0.255  -0.089  0.009  

-0.141  0.047  0.072  0.080  -0.060  -0.108  -0.044  -0.089  -0.005  0.068  -0.067  0.121  

-0.090  0.033  0.068  0.004  -0.017  0.072  0.006  0.111  0.013  -0.143  -0.082  0.186  

0.110  0.031  -0.030  -0.206  0.061  0.100  -0.049  0.005  -0.036  -0.169  -0.103  0.113  

0.145  -0.002  0.024  -0.170  0.058  -0.002  -0.107  -0.020  0.156  0.001  0.013  0.054  

-0.047  -0.108  0.020  -0.016  0.042  0.041  -0.007  -0.042  0.134  0.092  -0.030  -0.059  

-0.018  0.094  0.068  0.061  -0.057  0.062  -0.021  -0.135  -0.063  0.059  0.036  -0.179  

-0.103  0.031  -0.091  0.028  -0.080  0.157  0.043  -0.036  0.021  -0.088  0.107 

90  88  86  83  83  119  134  142  153  159  162  157  

90  85  83  77  78  112  126  135  143  149  151  151  

96  95  87  81  79  110  129  141  149  159  167  167  

97  98  91  87  85  117  136  147  159  185  190  184  

99  95  91  85  86  119  133  145  158  189  194  186  

105  94  92  87  88  117  130  145  160  191  194  186  

91  86  88  84  86  113  125  137  151  179  182  172  

84  81  83  79  78  104  118  127  142  168  169  160  

86  82  83  81  81  111  124  133  154  175  174  170  

90  90  91  89  92  120  136  145  167  188  187  184  

92  93  95  95  98  123  140  151  167  180  184  181  



80  72  70  70  65  66  63  63  67  76  80  85  

92  86  81  76  74  83  82  81  80  84  89  91  

121  115  114  112  109  115  111  113  112  116  116  118  

118  117  112  108  102  106  108  111  115  119  119  122  

103  98  95  93  94  91  93  96  99  104  109  110  

100  97  96  93  94  93  92  95  95  99  103  107  

108  107  112  114  114  116  114  111 

89  123  157  127  105  185  172  123  144  126  131  106  

59  104  123  137  122  162  174  156  161  137  73  32  

59  93  98  130  130  108  114  109  134  169  84  57  

123  167  154  146  134  94  88  88  158  204  124  104  

175  175  152  145  138  122  92  107  150  187  147  132  

174  147  117  117  117  112  90  83  128  151  123  120  

142  113  83  118  151  162  162  150  146  112  88  95  

139  90  47  112  182  218  215  168  94  105  137  135  

181  112  49  80  109  121  125  85  55  113  149  162  

212  169  122  114  117  114  126  84  85  139  177  200  



231  143  130  166  88  80  85  116  130  102  132  152  

150  135  71  50  67  117  169  218  238  128  127  143  

120  86  99  133  168  143  130  172  176  122  32  66  

152  150  104  96  177  136  126  171  167  150  54  114  

147  105  102  116  142  131  67  144  159  150  123  149  

116  112  98  125  133  104  54  98  77  85  111  144  

146  147  126  145  154  137  102  106  113  121  1

97  35  165  131  70  85  100  137  143  145  190  190  

138  108  186  143  137  184  181  143  60  124  117  126  

172  93  98  101  159  108  102  151  146  137  88  101  



176  130  100  160  139  138  109  144  95  163  91  152  

80  140  65  100  119  86  127  185  56  107  53  135  

108  108  151  174  185  120  116  165  154  181  138  247  

127  75  144  40  72  93  68  93  106  139  93  137  

209  130  175  137  147  211  164  123  155  185  97  66  

134  97  141  132  98  148  152  147  86  172  118  77  

94  136  127  106  109  94  126  160  94  172  153  144  

92  134  159  144  128  136  121  144  99  136  130  123  

110  145  168  165  158  122  92  123  105  119  103  122  

126  75  104  133  140  124  118  124  97  118  136  131  

187  73  108  181  102  118  191  193  126  105  155  147  

237  106  102  161  39  32  112  141  162  89  77  142  



112  113  125  135  83  148  208  176  168  80  79  92  

118  136  152  71  39  123  169  126  176  110  1

问题2——6D

In [22]:
import numpy as np
import os
import cv2
import shutil
import math
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

#手工实现PCA
# X代表训练集矩阵
# k代表需要保留的特征数量（即降维后的特征向量数量）
def pca(X,k):
    temp = k
    #矩阵行数代表样本数量n_samples，矩阵列数代表每个样本的特征数n_features(即特征向量)
    n_samples, n_features = X.shape
    #对矩阵进行零均值化，即减去这一行的均值
    mean = np.array([np.mean(X[:,i]) for i in range(n_features)])
    norm_X = X - mean

    #散度矩阵scatter_matrix
    #散度矩阵就是协方差矩阵*(总数据量-1),因此他们的特征根和特征向量是一样的
    scatter_matrix = np.dot(np.transpose(norm_X),norm_X)

    #计算特征向量和特征值
    eig_val, eig_vec = np.linalg.eig(scatter_matrix)
    eig_pairs = [(np.abs(eig_val[i]), eig_vec[:,i]) for i in range(n_features)]

    #根据特征值大小，从大到小对(特征值,特征向量)的多对pair进行排序
    eig_pairs.sort(reverse=True)
    #选出前k个特征向量
    feature=np.array([ele[1] for ele in eig_pairs[:k]])
    
    #输出降维后的特征向量
    print("降维后的特征向量为：\n")
    for k in range(6):
        for i in range(12):
            for j in range(12):
                print("%.3f  "%(feature[k][i*12+j]),end="")
            print("\n")
        print("\n")
    print("-----------------------------------------------------------------------------")

    #将特征向量rescale到0——255之间
    array = feature
    ymax = 255
    ymin = 0
    xmax = max(map(max,array))
    xmin = min(map(min,array))
    for i in range(6):
        for j in range(144):
            array[i][j] = int(round(((ymax-ymin)*(array[i][j]-xmin)/(xmax-xmin))+ymin))
    print("rescale到0—255范围内的特征向量为：\n")
    for k in range(6):
        for i in range(12):
            for j in range(12):
                print("%d  "%(array[k][i*12+j]),end="")
            print("\n")
        print("\n")
    print("-----------------------------------------------------------------------------")
    
    feature = np.array([ele[1] for ele in eig_pairs[:temp]])
    
    #Y = PX即为降维到k维后的数据
    data = np.dot(norm_X,np.transpose(feature))
    return data


#存储裁剪后图片的每个子图(12*12)
X = np.zeros((12,12),dtype = int)
#存储所有子图，组成训练集
X_new = np.zeros((144,144),dtype = int)

#读取代表图片
#img_num可修改为自己需要读取的图片编号
img_number = "915"
img = cv2.imread("C:/Users/93508/Desktop/Final/data/imgs/" + img_number + ".png")
#以灰度图的方式读取，去除图片的三通道RGB信息
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#将图片尺寸裁剪为12的倍数
img = cv2.resize(img,(144,144))

#将图片拆分12*12的子图并全部保存至X_new
#每个12*12的子图被视为144维的向量，将其一维扁平化存入X_new的每一行中
#由于图片为144*144，子图为12*12，所以X_new为144*144
#即144条数据，每条数据有144维的特征
for i in range(0,144):
    X = img[int(i/12)*12:int(i/12)*12+12,i%12*12:i%12*12+12]
    X_new[i,:] = X.reshape(1,-1)

#调用手工实现的PCA
result = pca(X_new,6)
print("压缩结果为：\n")
print(result)


降维后的特征向量为：

0.079  0.082  0.082  0.082  0.085  0.085  0.084  0.084  0.078  0.081  0.076  0.077  

0.066  0.068  0.069  0.071  0.073  0.072  0.073  0.074  0.071  0.073  0.068  0.071  

0.054  0.055  0.054  0.055  0.056  0.057  0.058  0.056  0.055  0.057  0.056  0.057  

0.072  0.070  0.066  0.068  0.070  0.070  0.071  0.070  0.066  0.069  0.068  0.067  

0.094  0.093  0.090  0.088  0.087  0.090  0.091  0.090  0.092  0.096  0.091  0.088  

0.102  0.102  0.100  0.100  0.098  0.098  0.099  0.100  0.103  0.105  0.100  0.096  

0.104  0.104  0.102  0.101  0.102  0.099  0.102  0.105  0.105  0.105  0.100  0.098  

0.089  0.089  0.088  0.089  0.091  0.089  0.090  0.091  0.093  0.092  0.088  0.087  

0.068  0.068  0.067  0.070  0.072  0.070  0.072  0.072  0.073  0.070  0.068  0.065  

0.076  0.074  0.073  0.073  0.075  0.075  0.076  0.078  0.076  0.075  0.073  0.073  

0.091  0.093  0.090  0.089  0.093  0.092  0.095  0.096  0.091  0.088  0.086  0.087  

0.090  0.094  0.091  0.091  0.093  0.095  

115  114  107  104  114  86  85  95  149  176  192  191  

69  74  71  65  73  49  54  74  120  159  167  173  



-----------------------------------------------------------------------------
压缩结果为：

[[ 5.33795462e+01 -6.14556979e+02 -4.54585915e+01 -2.63374140e+01
  -2.48860094e+02  1.54961254e+02]
 [-3.16548228e+02 -6.32657825e+01  1.64741630e+02 -1.42778766e+02
  -2.81498875e+02  9.69573659e+01]
 [-5.32292878e+02 -1.30667072e+02 -3.69134378e+01 -4.62395641e+00
   2.73603338e+01 -8.36798729e+01]
 [-6.14058375e+02  1.28210018e+02 -5.10475885e+01  2.22898170e+01
  -3.60726157e+01 -2.99281328e+01]
 [-1.60883017e+02 -4.18821598e+01 -1.85906614e+02  4.18976796e+01
   9.16141738e+01  5.61970072e+01]
 [-5.54760396e+02 -2.36622725e+02 -1.19408145e+02 -1.24837403e+02
  -6.43911072e+01  1.93034254e+01]
 [-1.67985847e+02  4.29166543e+02 -1.34332260e+00  1.31586749e+02
   1.66130961e+02 -9.62014383e+01]
 [ 3.03389126e+02 -3.48365541e+01 -2.10912428e+01  1.17638962e+02
  -1.58001800e+01  1.62342